# Dynamic World LULC Pipeline
The dynamic world land cover dataset is available through Google Earth Engine. 

[Dynamic World V1](https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1#terms-of-use)

[Introduction to Dynamic World Tutorials](https://developers.google.com/earth-engine/tutorials/community/introduction-to-dynamic-world-pt-1)

[Earth Engine Access](https://developers.google.com/earth-engine/guides/access)

## Imports

In [ ]:
# General imports:
import numpy as np
import rioxarray as rxr
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import mapping
import cartopy.crs as ccrs 
import os
import yaml

# Google Earth Engine imports:
import ee
import geemap
from google.auth import default
import certifi
import ssl

Prior to running the code below, I created an ee project. In this case, I initialized a free, non-commercial use project. This will need to be addressed as we move forward. The project was created in earth engine, and then the project ID and credentials were established by running the following line in terminal: 

`earthengine authenticate`

This will open a browser window to login to the appropriate Google account. In the case you are already logged into another account, run the following code to reset:

In [ ]:
# import os
# import shutil

# ee_credentials_path = os.path.expanduser("~/.config/earthengine/credentials")

# if os.path.exists(ee_credentials_path): 
#     shutil.rmtree(os.path.dirname(ee_credentials_path)) 
#     print("Existing Earth Engine credentials removed.") 
# else: 
#     print("No existing Earth Engine credentials found.")

Then if you are using a Mac, you may need to follow the instructions here to install your certificate: https://stackoverflow.com/questions/68275857/urllib-error-urlerror-urlopen-error-ssl-certificate-verify-failed-certifica

Following that, restart the kernel and continue:

In [ ]:
# Create a context using the certifi bundle.
context = ssl.create_default_context(cafile=certifi.where())

# Run the following code to authenticate. You will be sent to retrieve an API token.
ee.Authenticate()

# Initialize earth engine and geemap, replacing the project name below with your project name.
ee.Initialize(opt_url="https://earthengine.googleapis.com", project="dynamic-world-pipeline")
geemap.ee_initialize()


## Defining Pipeline Functions

In [ ]:
def get_boundaries(path):
    """Given path to geojson, return AOI as an ee Geometry object."""

    # Load the geojson.
    aoi_file = gpd.read_file(path)
    
    # Convert the GeoPandas geometry to GeoJSON.
    aoi_geom = aoi_file.iloc[0].geometry.__geo_interface__
    
    # Convert to an ee Geometry object.
    ee_polygon = ee.Geometry(aoi_geom)
        
    return(ee_polygon)

def get_bbox(path):
    """Given path to geojson, return AOI bounding box."""

    # Load the geojson.
    aoi_file = gpd.read_file(path)

    # Calculate the bounding box [minx, miny, maxx, maxy].
    bounds = aoi_file.total_bounds 
    
    # Create an ee.Geometry object from the bounding box.
    aoi = ee.Geometry.Rectangle([bounds[0], bounds[1], bounds[2], bounds[3]])
    
    return(aoi)

def fetch_dynamic_world(aoi_path, date, date_buffer, nodata_threshold, out_dir):
    """
    Function to acquire Dynamic World raster for a specific polygon and date range.
    
    Inputs:
    aoi         : ee.Geometry.Polygon defining the area of interest
    date        : specific date to aim for (YYYY-MM-DD)
    date_buffer : number of days to buffer date parameter with on either side
                  (e.g. start_date = date - date_buffer & 
                   end_date = date + date_buffer)
    
    Outputs:
    xr_ds : xarray Dataset containing the Dynamic World data composited over time period.
    """
    # Use date_buffer to set the start and end date surrounding date of interest.
    start_date = ee.Date(date).advance(-date_buffer, 'day')
    end_date = ee.Date(date).advance(date_buffer, 'day')
    
    # Get aoi bounding box polygon.
    aoi = get_bbox(aoi_path)
    
    # Load the Dynamic World image collection for the aoi and dates of interest.
    imcol = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
             .filterBounds(aoi)
             .filterDate(start_date, end_date)
             .select('label') # 'Label' contains the band index of highest probability land cover class for each pixel.
            )

    # Get the list of dates in the collection
    dates = imcol.aggregate_array('system:time_start').getInfo()
    dates = [np.datetime64(ee.Date(date).format('YYYY-MM-dd').getInfo()) for date in dates]
    print(f'Dynamic World Data Dates: {dates}')

    # If there is no data available, increase the date_buffer and re-run function.
    if len(dates) < 1:
        # If date_buffer exceeds 180, stop the recursion.
        if date_buffer >= 180:
            raise Exception(f"No Dynamic World data found within {date_buffer} days of the target date. Stopping recursion.")
        
        date_buffer += 15
        print(f"No dynamic world data is available within this time period. Increasing date buffer to: {date_buffer} and re-running.")

        # Re-run the function recursively with new date_buffer.
        return fetch_dynamic_world(aoi_path, date, date_buffer, nodata_threshold, out_dir)

    # Reduce to the most probable land cover type and calculate percent nodata.
    dw_composite = imcol.reduce(ee.Reducer.mode())
    pct_nodata = check_pct_null(dw_composite, aoi)
    
    # If the percent nodata is greater than the threshold, increase data_buffer and re-run function.
    if pct_nodata > nodata_threshold:
        # If date_buffer exceeds 180, stop the recursion.
        if date_buffer >= 180:
            raise Exception(f"Not enough Dynamic World data found within {date_buffer} days of the target date. Stopping recursion.")
        
        date_buffer += 15
        print(f"The amount of nodata in the image was: {pct_nodata}. Increasing date buffer to: {date_buffer} and re-running.")

        # Re-run the function recursively with new date_buffer.
        return fetch_dynamic_world(aoi_path, date, date_buffer, nodata_threshold, out_dir)
    
    # Otherwise, if there is enough data, export the composite.
    else:
        # Convert ee.Date to human-readable strings.
        start_date_str = start_date.format('YYYYMMdd').getInfo() 
        end_date_str = end_date.format('YYYYMMdd').getInfo()

        # Construct output file path.
        out_path = os.path.join(out_dir, f'dynamic_world_{start_date_str}_{end_date_str}.tif')
        
        # Export the dynamic world data as a GeoTIFF.
        geemap.ee_export_image(dw_composite, filename = out_path, scale = 10, region = aoi)
        print(f"Exported dynamic world composite to {out_path}")
        
        return out_path
    

def check_pct_null(image, aoi):
    # Get the internal mask of the image and invert so nodata is 1.
    nodata_mask = image.mask().Not()

    # Get the number of non-null pixels with .count().
    total_pixels = image.reduceRegion(
        reducer=ee.Reducer.count(), geometry=aoi, scale=10, maxPixels=1e8
    ).get('label_mode').getInfo()

    # Get the number of nodata pixels from the mask.
    nodata_pixels = nodata_mask.reduceRegion(
        reducer=ee.Reducer.sum(), geometry=aoi, scale=10, maxPixels=1e8
    ).get('label_mode').getInfo()

    # Calculate the percentage of nodata pixels.
    pct_nodata = (nodata_pixels / (total_pixels + nodata_pixels)) * 100

    return pct_nodata

## Running Pipeline

In [ ]:
# Set target date of interest.
date = '2023-06-01'
date_buffer = 1
nodata_threshold = 100

# Read config information from yml file.
with open("config.yml", "r") as yml:
    config = yaml.safe_load(yml)

out_dir = config['out-dir']
aoi_path = config['aoi-path']

# Call function to export dynamic world raster.
dynamic_world_tif = fetch_dynamic_world(aoi_path, date, date_buffer, nodata_threshold, out_dir)

## Plotting outputs

In [ ]:
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap, BoundaryNorm

# Define the color palette, labels, and corresponding values.
colors = ['#419bdf', '#397d49', '#88b053', '#7a87c6', '#e49635', '#dfc35a', '#c4281b', '#a59b8f', '#B39FE1']  
labels = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
values = [0, 1, 2, 3, 4, 5, 6, 7, 8] 

# Create a colormap and a normalization based on the values.
cmap = ListedColormap(colors)
norm = BoundaryNorm(values + [max(values) + 1], cmap.N)  # Ensures correct mapping of values to colors

# Open the GeoTIFF using rioxarray.
classification = rxr.open_rasterio(dynamic_world_tif, masked=True)

# Plot the dynamic world composite raster.
plt.figure()
img = classification.plot(cmap=cmap, norm=norm, add_colorbar=False)
img.axes.set_aspect('equal')

# Read the AOI geojson as geodataframe.
aoi_gdf = gpd.read_file(aoi_path)
aoi_gdf.boundary.plot(ax=img.axes, edgecolor='black', linewidth=2)

# Create a custom legend.
patches = [mpatches.Patch(color=colors[i], label=labels[i]) for i in range(len(labels))]
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot.
plt.title('Land Cover Classification')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap, BoundaryNorm
import rioxarray as rxr

tif_path = '/Users/snix/Downloads/composite_2021-06-05.tif'

# Define colors, labels, and values.
colors = ['#419bdf', '#397d49', '#88b053', '#7a87c6', '#e49635', '#dfc35a', '#c4281b', '#a59b8f', '#B39FE1']
labels = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']
values = list(range(len(labels)))  # 0–8

# Create colormap and normalization.
cmap = ListedColormap(colors)
norm = BoundaryNorm(values + [max(values) + 1], cmap.N)

# Open the raster.
classification = rxr.open_rasterio(tif_path, masked=True)

# 'label' band is band 10
if 'band' in classification.dims:
    classification = classification.sel(band=10)

# Plot the raster.
plt.figure()
img = classification.plot(cmap=cmap, norm=norm, add_colorbar=False)
img.axes.set_aspect('equal')

# Create a legend.
patches = [mpatches.Patch(color=colors[i], label=labels[i]) for i in range(len(labels))]
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('Land Cover Classification')
plt.show()
